In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd

neg_rev = pd.read_csv('/content/drive/MyDrive/Team#2_LDA/neg_review.csv')
pos_rev = pd.read_csv('/content/drive/MyDrive/Team#2_LDA/pos_review.csv')

pos_rev.head(5)

,Unnamed: 0,review
0,0,nice hotel expensive parking got good deal sta...
1,1,"unique, great stay, wonderful time hotel monac..."
2,2,"great stay great stay, went seahawk game aweso..."
3,3,love monaco staff husband stayed hotel crazy w...
4,4,"cozy stay rainy city, husband spent 7 nights m..."


In [ ]:
# Word appearance rate in text as war_text
# Relativity imporatnce in text as ri_text
def LDA_model(dataframe, columns):
  temp_reviews = dataframe[columns].tolist()
  
  LDA_vetorizer = CountVectorizer(strip_accents = 'unicode',
                                  stop_words = 'english',
                                  lowercase = True,
                                  token_pattern = r'\b[a-zA-Z]{3,}\b',
                                  max_df = 0.9,
                                  min_df = 10)
  
  tfid_vectorizer = TfidVectorizer(**LDA_vetorizer.get_params())
  mat_review = tfid_vectorizer.fit_transform(temp_reviews)